### Imports and database initialization

In [2]:
from sqlalchemy import select, func
from gorillatracker.ssl_pipeline.models import Tracking, Video, Camera
from sqlalchemy.orm import sessionmaker
from gorillatracker.ssl_pipeline.dataset import GorillaDatasetKISZ

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter

import numpy as np

engine = GorillaDatasetKISZ().engine
session_cls = sessionmaker(bind=engine)

### Number of Videos with x tracklets (174 outliers not shown)

In [ ]:
stmt = (
    select(Video.video_id)
)

with session_cls() as session:
    result = session.execute(stmt).fetchall()

num_videos = len(result)

stmt = (
    select(Tracking.video_id, func.count(Tracking.tracking_id).label("count"))
    .group_by(Tracking.video_id)
    )

with session_cls() as session:
    result = session.execute(stmt)
    sorted_all = sorted(result.fetchall(), key=lambda x: x[1], reverse=True)

num_videos_no_tracking = num_videos - len(sorted_all)

res = {}

for video_id, count in sorted_all:
    if count not in res:
        res[count] = 1
    res[count] += 1

res[0] = num_videos_no_tracking

count = sum([res[x] for x in res.keys() if x > 40])
print(count)
print(sorted([(x, res[x]) for x in res.keys() if x > 40]))

res = res.items()

list1, list2 = zip(*res)

plt.bar(list1, list2)
plt.xlabel("Tracklet count")
plt.ylabel("Number of videos")
plt.xlim(-1,40)
plt.title("Number of videos with x tracklets")
plt.tight_layout()
plt.show()

### Tracklet distribution heatmap over camera positions

In [ ]:
from ipywidgets import IntProgress

stmt = (
    select(Video.camera_id, func.sum(Tracking.tracking_id).label("count"))
    .join(Tracking, Video.video_id == Tracking.video_id)
    .group_by(Video.camera_id)
    )

with session_cls() as session:
    result = session.execute(stmt)
    all = sorted(result.fetchall(), key=lambda x: x[0])

stmt = (
    select(Camera.camera_id, Camera.longitude, Camera.latitude)
    )

with session_cls() as session:
    result = session.execute(stmt)
    all2 = sorted(result.fetchall(), key=lambda x: x[0])

mapping = {x : (y,z) for x, y, z in all2}

f = IntProgress(min=0, max=len(all)) # instantiate the bar
display(f)

points = []
for (camera_id, count) in all:
    f.value += 1
    (long, lat) = mapping[camera_id]
    if long is None or lat is None:
        continue
    for i in range(count//10000):
        points.append((long + np.random.normal(0, 0.002), lat + np.random.normal(0, 0.002)))

x, y = zip(*points)

plt.hist2d(x,y, bins=1000, cmap="twilight")
plt.xlabel("Longitude (2.17km between ticks)")
plt.ylabel("Latitude (2.22km between ticks)")
plt.title("Tracklet distribution heatmap over camera positions")
for i, (x, y) in mapping.items():
    if x is None or y is None:
        continue
    plt.text(x, y, f"{i}", ha="center", fontsize=6)

plt.show()

### Number of Videos per Camera

In [ ]:
stmt = (
    select(Video.camera_id, func.count(Video.video_id).label("count"))
    .group_by(Video.camera_id)
)

with session_cls() as session:
    result = session.execute(stmt)
    result_list = result.fetchall()

sorted_all = sorted(result_list, key=lambda x: x.count, reverse=True)

list1, list2 = zip(*[(row.camera_id, row.count) for row in sorted_all])

plt.xticks(rotation=90, fontsize=4)
plt.bar(range(len(list1)), list2, tick_label=list1)
plt.xlabel("Camera ID")
plt.ylabel("Number of videos")
plt.title("Number of videos per camera")
plt.tight_layout()
plt.show()

### Length of Videos per Camera

In [ ]:
stmt = (
    select(Video.camera_id, func.avg((Video.frames / Video.fps).label("duration")).label("count"))
    .group_by(Video.camera_id)
)

with session_cls() as session:
    result = session.execute(stmt)
    result_list = result.fetchall()

sorted_all = sorted(result_list, key=lambda x: x.count, reverse=True)

list1, list2 = zip(*[(row.camera_id, row.count) for row in sorted_all])

plt.xticks(rotation=90, fontsize=4)
plt.bar(range(len(list1)), list2, tick_label=list1)
plt.xlabel("Camera ID")
plt.ylabel("Average video duration (s)")
plt.title("Average video length per camera")
plt.tight_layout()
plt.show()

### Length of Videos

In [ ]:
stmt = (
    select((Video.frames / Video.fps).label("duration"), func.count(Video.video_id).label("count"))
    .group_by("duration")
    )

with session_cls() as session:
    result = session.execute(stmt)
    all = result.fetchall()

res = {}

bins = 10
for length, count in all:
    if length // bins * bins not in res:
        res[length // bins * bins] = 0
    res[length // bins * bins] += count

list1, list2 = zip(*res.items())
plt.bar(list1, list2, width=5)
plt.xlabel(f"Length (seconds) in bins of {bins}")
plt.ylabel("Number of Videos (log scale)")
plt.yscale("log")
plt.title("Number of videos per length")
plt.tight_layout()
plt.show()

### Distribution of Video Recording Start Times

In [ ]:
from datetime import datetime

stmt = (
    select(Video.start_time)
)

with session_cls() as session:
    result = session.execute(stmt)
    all = result.fetchall()
    all_times = [mdates.date2num(datetime.combine(datetime.today(), t[0].time())) for t in all if t[0] is not None]
    
fig, ax = plt.subplots()
ax.hist(all_times, bins=24 * 4, rwidth=0.8)
ax.xaxis.set_major_formatter(DateFormatter("%H:%M"))
plt.xlabel("Time of Day Start Time")
plt.ylabel("Number of Videos")
plt.title("Distribution of Video Recording Times")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Number of Videos with and without Camera Position per Date (stacked graph)

In [ ]:
stmt = (
    select(Video.start_time).join(
        Camera, Video.camera_id == Camera.camera_id
        ).where(Video.start_time != None).where(Camera.longitude == None)
    )

with session_cls() as session:
    result = session.execute(stmt)
    no_pos = result.fetchall()
    
stmt = (
    select(Video.start_time).join(
        Camera, Video.camera_id == Camera.camera_id
        ).where(Video.start_time != None).where(Camera.longitude != None)
    )

with session_cls() as session:
    result = session.execute(stmt)
    with_pos = result.fetchall()
    
no_pos_dates = [mdates.date2num(date[0].date()) for date in no_pos if date[0] is not None and date[0].date().year > 2000]
with_pos_dates = [mdates.date2num(date[0].date()) for date in with_pos if date[0] is not None and date[0].date().year > 2000]

fig, ax = plt.subplots()
plt.hist([no_pos_dates, with_pos_dates], bins=100, color=['blue', 'green'], stacked=True)
plt.legend(["Without Camera Position", "With Camera Position"])
ax.xaxis.set_major_formatter(DateFormatter("%Y-%m-%d"))
ax.xaxis.set_major_locator(mdates.YearLocator())
plt.xlabel("Date")
plt.ylabel("Number of Videos")
plt.xticks(rotation=45)
plt.title("Number of videos per date")
plt.tight_layout()
plt.show()